In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install -U py-boost

  Obtaining dependency information for py-boost from https://files.pythonhosted.org/packages/35/fe/8ffba87d7701b9952e44d3e275aa682a3bbdb5179381516082b91828fc85/py_boost-0.4.3-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 2.9 MB/s eta 0:00:00


In [43]:
import cupy as cp
from py_boost.gpu.utils import *
from astropy.table import Table
from cupyx.profiler import benchmark

In [32]:
def sample_idx(n, sample):
    # THIST FUNCTION GENERATES IDS USED 
    # IN THE HISTOGRAM CALCULATIONS
    
    idx = cp.arange(n, dtype=cp.uint64)
    sl = cp.random.rand(n) < sample
    
    return cp.ascontiguousarray(idx[sl])

def generate_input( n_rows, n_cols, n_out, max_bin, nnodes, 
                    colsample=0.8, subsample=0.8, outsample=1.0, verbose=False, seed=42):
    # THIS FUNCTION GENERATES ALL INPUT
    # ARRAYS, REQUIRED BY THE HISTOGRAM 
    # FUNCTION IN PY-BOOST
    # Input: 
    # n_rows   - number of rows in the input array
    # n_cols   - number of cols in the input array
    # n_out    - number of ???? in the output array
    # max_bins - number of histogram bins (can't be >256 really)
    # nnodes   - ????
    
    np.random.seed(seed)
    features_cpu = np.random.randint(0, max_bin, size=(n_rows, n_cols)).astype(np.uint8)
    features_gpu = pad_and_move(features_cpu)
    cp.random.seed(seed)
    targets_gpu  = cp.random.rand(n_rows, n_out).astype(np.float32)
    cp.random.seed(seed)
    nodes_gpu    = cp.random.randint(0, nnodes, size=(n_rows, )).astype(np.int32)
    cp.random.seed(seed)
    
    if verbose == True:
        print('Initial CPU features shape: {}'.format(features_cpu.shape))
        print('Padded  GPU features shape: {}'.format(features_gpu.shape))
        print('Nodes   GPU vector   shape: {}'.format(nodes_gpu.shape   ))
        print('Targets GPU array    shape: {}'.format(targets_gpu.shape ))
    
    row_indexer = sample_idx(n_rows, subsample)
    col_indexer = sample_idx(n_cols, colsample)
    out_indexer = sample_idx(n_out, outsample)
    
    if verbose == True:
        print('Sampled rows shape:    {}'.format(row_indexer.shape))
        print('Sampled columns shape: {}'.format(col_indexer.shape))
        print('Sampled output shape:  {}'.format(out_indexer.shape))
    
    nout   = out_indexer.shape[0]
    nfeats = col_indexer.shape[0]
    
    # Anton's function takes the following input arguments + the empty array to 
    # store the resulting histogram bins (comes on the first position)
    # input: res, X, Y, nodes, col_indexer, row_indexer, out_indexer
    
    res    = cp.zeros((nout, nnodes, nfeats, max_bin), dtype=cp.float32)
    params = (res, features_gpu, targets_gpu, nodes_gpu, col_indexer, row_indexer, out_indexer)
    
    if verbose == True:
        print ('Sum of the resulting histogram must be {}'.format(nfeats * targets_gpu[row_indexer].sum()))
    return params

In [41]:
input_params = generate_input(n_rows=pow(10,6),n_cols=99,n_out=10,max_bin=256,nnodes=32,verbose=True) 

Initial CPU features shape: (1000000, 99)
Padded  GPU features shape: (1000000, 99)
Nodes   GPU vector   shape: (1000000,)
Targets GPU array    shape: (1000000, 10)
Sampled rows shape:    (800346,)
Sampled columns shape: (74,)
Sampled output shape:  (10,)
Sum of the resulting histogram must be 296096096.0


In [44]:
res = input_params[0].copy()
fill_histogram( res, *input_params[1:] )
print (res.sum())

296096130.0


In [63]:
# Check the performance of the algorithm for different values of n_rows (time is in microseconds)
tab_out = Table()
fname   = 'tab_Anton_Nrows_vs_time.txt'
fmts    = {'N_rows'   :  '%.2f',
           'tau_mean' :  '%.6f',
           'tau_stdev':  '%.6f'}

exponent    = np.arange(2,6.1,0.25)    
n_rows      = np.array([pow(10,x) for x in exponent],dtype=np.int32)
n_threads   = 1024
to_microsec = pow(10,6)

ns, means, stdevs = [],[],[]
for n in n_rows:
    input_params = generate_input(n_rows=n,n_cols=99,n_out=10,max_bin=256,nnodes=32,verbose=False)
    tau = benchmark( fill_histogram, input_params, n_repeat=100, n_warmup=10 )
    
    mean, stdev = np.average(tau.gpu_times), np.std(tau.gpu_times)
    
    print ('N_rows = 10^{:.2f}'.format(np.log10(n)))
    #print (tau)
    print ('time   = {:.6f}+/-{:.6f} microsec'.format(mean*to_microsec, stdev*to_microsec))

    ns.append( np.log10(n) )
    means.append(mean*to_microsec)
    stdevs.append(stdev*to_microsec)
    
tab_out['N_rows']    = ns
tab_out['tau_mean']  = means
tab_out['tau_stdev'] = stdevs

tab_out.write( fname, format='ascii.fixed_width', formats=fmts, bookend=False, delimiter=None, overwrite=True )

N_rows = 10^2.00
time   = 1896.876161+/-114.624658 microsec
N_rows = 10^2.25
time   = 1828.759998+/-110.347305 microsec
N_rows = 10^2.50
time   = 2019.174410+/-231.365134 microsec
N_rows = 10^2.75
time   = 2013.082236+/-395.090508 microsec
N_rows = 10^3.00
time   = 1823.944961+/-142.058531 microsec
N_rows = 10^3.25
time   = 1859.663681+/-161.301253 microsec
N_rows = 10^3.50
time   = 1823.477436+/-152.702139 microsec
N_rows = 10^3.75
time   = 1811.499839+/-116.196733 microsec
N_rows = 10^4.00
time   = 1849.626563+/-155.482864 microsec
N_rows = 10^4.25
time   = 1885.572470+/-167.035890 microsec
N_rows = 10^4.50
time   = 2090.731832+/-171.852686 microsec
N_rows = 10^4.75
time   = 2693.894415+/-159.887266 microsec
N_rows = 10^5.00
time   = 5152.848954+/-248.592787 microsec
N_rows = 10^5.25
time   = 7995.912633+/-835.150112 microsec
N_rows = 10^5.50
time   = 14364.097929+/-2120.450877 microsec
N_rows = 10^5.75
time   = 26390.166340+/-3222.756113 microsec
N_rows = 10^6.00
time   = 46374.6489

In [67]:
# Check the performance of the algorithm for different values of n_cols (time is in microseconds)
tab_out = Table()
fname   = 'tab_Anton_Ncols_vs_time.txt'
fmts    = {'N_cols'   :  '%d',
           'tau_mean' :  '%.6f',
           'tau_stdev':  '%.6f'}
  
n_cols      = np.arange(50, 1051, 100)
n_threads   = 1024
to_microsec = pow(10,6)

ns, means, stdevs = [],[],[]
for n in n_cols:
    input_params = generate_input(n_rows=pow(10,6),n_cols=n,n_out=10,max_bin=256,nnodes=32,verbose=False)
    tau = benchmark( fill_histogram, input_params, n_repeat=100, n_warmup=10 )
    
    mean, stdev = np.average(tau.gpu_times), np.std(tau.gpu_times)
    
    print ('N_cols = {}'.format(n))
    print (tau)
    print ('time   = {:.6f}+/-{:.6f} microsec'.format(mean*to_microsec, stdev*to_microsec))

    ns.append( n )
    means.append(mean*to_microsec)
    stdevs.append(stdev*to_microsec)
    
tab_out['N_cols']    = ns
tab_out['tau_mean']  = means
tab_out['tau_stdev'] = stdevs

tab_out.write( fname, format='ascii.fixed_width', formats=fmts, bookend=False, delimiter=None, overwrite=True )

N_cols = 50
fill_histogram      :    CPU: 22638.828 us   +/- 1454.869 (min: 20710.018 / max: 25962.042) us     GPU-0: 22655.140 us   +/- 1455.321 (min: 20721.760 / max: 25978.527) us
time   = 22655.140152+/-1455.321176 microsec
N_cols = 150
fill_histogram      :    CPU: 77966.267 us   +/- 7431.712 (min: 71762.934 / max: 138324.910) us     GPU-0: 77987.432 us   +/- 7431.432 (min: 71780.350 / max: 138346.497) us
time   = 77987.431870+/-7431.432292 microsec
N_cols = 250
fill_histogram      :    CPU: 142058.543 us   +/- 6109.221 (min: 131812.327 / max: 175843.713) us     GPU-0: 142086.368 us   +/- 6109.532 (min: 131849.792 / max: 175874.268) us
time   = 142086.367798+/-6109.531662 microsec
N_cols = 350
fill_histogram      :    CPU: 204011.269 us   +/- 3225.648 (min: 192993.902 / max: 216768.507) us     GPU-0: 204045.095 us   +/- 3225.689 (min: 193023.972 / max: 216799.744) us
time   = 204045.094604+/-3225.689304 microsec
N_cols = 450
fill_histogram      :    CPU: 265058.949 us   +/- 4007.8

In [68]:
# Check the performance of the algorithm for different values of n_bins (time is in microseconds)
tab_out = Table()
fname   = 'tab_Anton_Nmaxbins_vs_time.txt'
fmts    = {'N_mbins'  :  '%d',
           'tau_mean' :  '%.6f',
           'tau_stdev':  '%.6f'}
  
n_bins      = np.arange(8,260,8) 
n_threads   = 1024
to_microsec = pow(10,6)

ns, means, stdevs = [],[],[]
for n in n_bins:
    input_params = generate_input(n_rows=pow(10,6),n_cols=99,n_out=10,max_bin=n,nnodes=32,verbose=False)
    tau = benchmark( fill_histogram, input_params, n_repeat=100, n_warmup=10 )
    
    mean, stdev = np.average(tau.gpu_times), np.std(tau.gpu_times)
    
    print ('N_cols = {}'.format(n))
    print (tau)
    print ('time   = {:.6f}+/-{:.6f} microsec'.format(mean*to_microsec, stdev*to_microsec))

    ns.append( n )
    means.append(mean*to_microsec)
    stdevs.append(stdev*to_microsec)
    
tab_out['N_mbins']   = ns
tab_out['tau_mean']  = means
tab_out['tau_stdev'] = stdevs

tab_out.write( fname, format='ascii.fixed_width', formats=fmts, bookend=False, delimiter=None, overwrite=True )

N_cols = 8
fill_histogram      :    CPU: 45462.434 us   +/- 1126.611 (min: 43781.185 / max: 50423.162) us     GPU-0: 45480.149 us   +/- 1126.295 (min: 43798.496 / max: 50443.295) us
time   = 45480.148773+/-1126.294500 microsec
N_cols = 16
fill_histogram      :    CPU: 43186.079 us   +/- 3989.742 (min: 40323.093 / max: 69160.232) us     GPU-0: 43202.815 us   +/- 3990.174 (min: 40341.377 / max: 69177.635) us
time   = 43202.814980+/-3990.174126 microsec
N_cols = 24
fill_histogram      :    CPU: 43879.011 us   +/- 4300.323 (min: 40774.849 / max: 73107.095) us     GPU-0: 43897.231 us   +/- 4300.370 (min: 40792.065 / max: 73125.443) us
time   = 43897.231293+/-4300.369872 microsec
N_cols = 32
fill_histogram      :    CPU: 43313.278 us   +/- 3193.374 (min: 40734.188 / max: 64033.917) us     GPU-0: 43331.005 us   +/- 3194.200 (min: 40749.985 / max: 64059.425) us
time   = 43331.004829+/-3194.200362 microsec
N_cols = 40
fill_histogram      :    CPU: 42275.667 us   +/- 3898.501 (min: 39833.559 / m

In [73]:
# Check the performance of the algorithm for different values of n_nodes (time is in microseconds)
tab_out = Table()
fname   = 'tab_Anton_Nnodes_vs_time.txt'
fmts    = {'N_nodes'  :  '%d',
           'tau_mean' :  '%.6f',
           'tau_stdev':  '%.6f'}
  
n_nodes     = np.arange(8,260,8) 
n_threads   = 1024
to_microsec = pow(10,6)

ns, means, stdevs = [],[],[]
for n in n_nodes:
    input_params = generate_input(n_rows=pow(10,6),n_cols=99,n_out=10,max_bin=256,nnodes=n,verbose=False)
    tau = benchmark( fill_histogram, input_params, n_repeat=100, n_warmup=10 )
    
    mean, stdev = np.average(tau.gpu_times), np.std(tau.gpu_times)
    
    print ('N_cols = {}'.format(n))
    print (tau)
    print ('time   = {:.6f}+/-{:.6f} microsec'.format(mean*to_microsec, stdev*to_microsec))

    ns.append( n )
    means.append(mean*to_microsec)
    stdevs.append(stdev*to_microsec)
    
tab_out['N_nodes']   = ns
tab_out['tau_mean']  = means
tab_out['tau_stdev'] = stdevs

tab_out.write( fname, format='ascii.fixed_width', formats=fmts, bookend=False, delimiter=None, overwrite=True )

N_cols = 8
fill_histogram      :    CPU: 42540.406 us   +/- 2623.313 (min: 39807.475 / max: 53910.224) us     GPU-0: 42557.898 us   +/- 2623.980 (min: 39822.849 / max: 53928.288) us
time   = 42557.897644+/-2623.980207 microsec
N_cols = 16
fill_histogram      :    CPU: 44159.171 us   +/- 3277.674 (min: 40738.146 / max: 62318.394) us     GPU-0: 44176.796 us   +/- 3277.405 (min: 40756.702 / max: 62336.609) us
time   = 44176.795616+/-3277.404812 microsec
N_cols = 24
fill_histogram      :    CPU: 48835.963 us   +/- 4752.256 (min: 44674.598 / max: 81782.409) us     GPU-0: 48856.109 us   +/- 4752.120 (min: 44693.214 / max: 81799.553) us
time   = 48856.109085+/-4752.119571 microsec
N_cols = 32
fill_histogram      :    CPU: 50352.921 us   +/- 5069.238 (min: 46406.794 / max: 86787.256) us     GPU-0: 50371.946 us   +/- 5069.816 (min: 46424.095 / max: 86818.787) us
time   = 50371.945915+/-5069.815794 microsec
N_cols = 40
fill_histogram      :    CPU: 50642.113 us   +/- 5105.341 (min: 46357.586 / m